In [ ]:
import ROOT
import numpy as np

In [ ]:

def read(infile):
	"""
	Open the ROOT file and get the data saved by AnalsyisData.C
	"""
	root_file = ROOT.TFile(infile, "READ")
	keys = root_file.Get("keys")
	keys = [str(key) for key in keys]
	data_dict = {	key : list(root_file.Get(key)) for key in keys	}

	return data_dict

def read_matrix(infile):
	"""
	Open the txt file and get the matrix
	the first line is the number of rows and columns
	"""
	with open(infile, "r") as f:
		lines = f.readlines()
		header = lines[0].split()
		nrows = int(header[0])
		ncols = int(header[1])
		matrix = np.array([[float(x) for x in line.split()] for line in lines[1:]])
		matrix = matrix[1:, 1:]

	print("Read matrix with shape: ", nrows, ncols)
	return matrix

def get_diff(list1, list2):
	"""
	Get the difference between two lists
	"""
	return [x-y for x,y in zip(list1, list2)]

def cal_chi2(data, theory, cov_matrix):
	"""
	Calculate the chi2 given the covaraice matrix
	"""
	data_ = np.array(data)		#protect the original data
	theory_ = np.array(theory)	#protect the original theory

	diff = np.array(get_diff(data_, theory_))
	cov_matrix_inv = np.linalg.inv(np.array(cov_matrix))
	chi2 = diff.T.dot(cov_matrix_inv).dot(diff)

	return chi2

def linear_func(x, a, b):
	"""
	The function to fit
	"""
	return a*x + b


In [ ]:

def minimize_chi2(x, y, cov_matrix):
	"""
	Minimize the chi2 given the covariance matrix
	"""
	a_width = 2.0e-05
	a_step 	= 1.0e-06
	b_width = 2.0e-02
	b_step 	= 1.0e-05

	a_n_step = int(2*a_width/a_step)
	b_n_step = int(2*b_width/b_step)
	print("a_n_step: ", a_n_step, "b_n_step: ", b_n_step)


	x = np.array(x)
	y = np.array(y)

	# starting values for the best fit
	a_best, b_best = np.polyfit(x, y, 1)
	chi2_best = cal_chi2(y, linear_func(x, a_best, b_best), cov_matrix)
	print(f"{'Starting: ':<15} a = {a_best:<15.2e}, b = {b_best:<15.2e}, chi2 = {chi2_best:<15.2e}")

	a_start = a_best - a_width
	b_start = b_best - b_width
	a_run = a_start
	b_run = b_start
	for i in range(a_n_step):
		for j in range(b_n_step):
			chi2_run = cal_chi2(y, linear_func(x, a_run, b_run), cov_matrix)
			if chi2_run < chi2_best:
				chi2_best = chi2_run
				a_best = a_run
				b_best = b_run

			# print(f"{'Running: ':<15} a = {a_run:<15.2e}, b = {b_run:<15.2e}, chi2 = {chi2_run:<15.2e}")
			b_run += b_step
		b_run = b_start
		a_run += a_step

	print(f"{'Best fit: ':<15} a = {a_best:<15.2e}, b = {b_best:<15.2e}, chi2 = {chi2_best:<15.2e}")
	return a_best, b_best, chi2_best

cov_matrix = read_matrix("outFiles/CovMatrix.txt")
data = read("outFiles/Result_CMS.root")
x = data["W"]
y = data["Sigma"]
y_sorted = [x for _, x in sorted(zip(x, y))]
x_sorted = sorted(x)
x_fit = np.array(x_sorted[1:])
y_fit = np.array(y_sorted[1:])

print(x_fit)
print(y_fit)

a, b, chi2 = minimize_chi2(x_fit, y_fit, cov_matrix)


In [ ]:
from matplotlib import pyplot as plt
from iminuit import Minuit

def chi2(a, b):
	"""
	Calculate the chi2 given the covaraice matrix
	"""
	data_ = np.array(y_fit)		#protect the original data
	theory_ = np.array(linear_func(x_fit, a, b))	#protect the original theory

	diff = np.array(get_diff(data_, theory_))
	cov_matrix_inv = np.linalg.inv(np.array(cov_matrix))
	chi2 = diff.T.dot(cov_matrix_inv).dot(diff)

	return chi2

m = Minuit(chi2, a=2.2e-5, b=2e-2)
# m.errors = (1e-10, 1e-10)
m.migrad()


In [ ]:
print(m.values)
print(f"{'Best fit: ':<15} a = {m.values['a']:<15.2e} , b = {m.values['b']:<15.2e}, chi2 = {m.fval:<15.2e}, chi2/ndf = {m.fval/(len(x_fit)-2):<15.2e}")

plt.plot(x_fit, y_fit, "o")
plt.plot(x_fit, linear_func(x_fit, *m.values))
plt.yscale("log")
plt.xlim(0,430)
plt.ylim(0.01, 0.1)